In [3]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017/")
db = client["mistral"]

In [5]:
db.list_collection_names()

['mistral_7b_instruct_v0.3_gguf:latest']

In [27]:
def extract_label(answer):
    answer = str(answer).strip()
    if answer.lower().startswith('true'):
        return True
    elif answer.lower().startswith('false'):
        return False
    else:
        return None

In [6]:
collection = db[db.list_collection_names()[0]]

# Fetch first 10 documents
docs = collection.find().limit(10)

In [7]:
databases = client.list_database_names()
databases

['admin', 'config', 'local', 'mistral', 'phi4']

In [20]:
docs = collection.find({}, {"ground_truth": 1, "answer": 1, "_id": 0})  # _id:0 excludes the default id field

In [21]:
results = [i for i in docs]

In [22]:
import pandas as pd

In [23]:
df = pd.DataFrame(results)

In [24]:
df.head()

,ground_truth,answer
0,True,False. While Persian (Farsi) is an official l...
1,True,True
2,True,True
3,True,True
4,False,False


In [28]:
df['pred'] = df['answer'].apply(extract_label)

In [29]:
df['pred'] = df['pred'].fillna('UNK')

In [30]:
df['pred'].value_counts(dropna=False)

pred
True     5692
False    3660
UNK        75
Name: count, dtype: int64

In [31]:
pd.crosstab(df['pred'], df['ground_truth'])

ground_truth,False,True
pred,,
False,2851,809
True,673,5019
UNK,29,46


In [33]:
df_filtered = df.copy()
df_filtered = df_filtered[df_filtered['pred'].isin([True, False])].copy()
df_filtered['pred'] = df_filtered['pred'].astype(bool)

In [35]:
from sklearn.metrics import classification_report

In [36]:
df_filtered['ground_truth'].value_counts(dropna=False)

ground_truth
True     5828
False    3524
Name: count, dtype: int64

In [37]:
df_filtered['pred'].value_counts(dropna=False)

pred
True     5692
False    3660
Name: count, dtype: int64

In [38]:
# Generate report
report = classification_report(
    y_true=df_filtered['ground_truth'],
    y_pred=df_filtered['pred'],
    target_names=['False', 'True'],
    zero_division=0
)
print(report)

              precision    recall  f1-score   support

       False       0.78      0.81      0.79      3524
        True       0.88      0.86      0.87      5828

    accuracy                           0.84      9352
   macro avg       0.83      0.84      0.83      9352
weighted avg       0.84      0.84      0.84      9352

